In [ ]:
import os
from os import path
from astropy.time import Time
from astropy.io import fits, ascii
import astropy.units as u
from astropy.table import Table, QTable
from astropy.constants import G

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import h5py
from sqlalchemy import func
from sqlalchemy.orm.exc import NoResultFound
from scipy.optimize import root
from scipy.stats import scoreatpercentile
import tqdm
import schwimmbad
from scipy.optimize import root
import pandas as pd

from thejoker import JokerSamples
from thejoker.sampler import JokerParams, TheJoker
from thejoker.plot import plot_rv_curves

from twoface.config import TWOFACE_CACHE_PATH
from twoface.data import APOGEERVData
from twoface.plot import plot_data_orbits, plot_two_panel
from twoface.mass import get_m2_min, mf
from twoface.ext import FileLoader

from apogeebh.db import db_connect, AllStar, AllVisit, StarResult, Status
from apogeebh.plot import plot_diag_panel

In [ ]:
joker_pars = JokerParams(P_min=1 * u.day,
                         P_max=1024 * u.day,
                         jitter=150 * u.m/u.s)

In [ ]:
sheet = FileLoader.GoogleSheets('1z0ukn8QWJL7dTrPdCFWlhQ6r-X-kwiA4z0YRqalZsY4', 1633658731)
dr14_candidates = sheet.load()

In [ ]:
Session, _ = db_connect('../cache/apogeebh.sqlite')
session = Session()

cache_file = '../cache/apogeebh-joker.hdf5'

In [ ]:
for i in range(5):
    n = session.query(AllStar).join(StarResult).filter(StarResult.status_id == i).count()
    print('status={0}: {1}'.format(i, n))

In [ ]:
m1 = 1.3*u.Msun
m2_cut = 1.4*u.Msun

apogee_ids = []
m2_filter = []
with h5py.File(cache_file, 'r') as f:
    for apogee_id in tqdm.tqdm(f):
        star = AllStar.get_apogee_id(session, apogee_id)
        samples = JokerSamples.from_hdf5(f[apogee_id])
            
        mfs = mf(samples['P'], samples['K'], samples['e'])
        m1s = [m1.value]*len(mfs) * m1.unit
        m2_min = get_m2_min(m1s, mfs)
        m2_mask = np.percentile(m2_min.value, 15)*m2_min.unit > m2_cut

        apogee_ids.append(apogee_id)
        m2_filter.append(m2_mask)

m2_filter = np.array(m2_filter)
apogee_ids = np.array(apogee_ids)

In [ ]:
new_candidates = apogee_ids[(~np.isin(apogee_ids, dr14_candidates['APOGEE_ID'])) & m2_filter]
m2_filter.sum(), len(new_candidates)

In [ ]:
with h5py.File(cache_file, 'r') as f:
    for apogee_id in tqdm.tqdm(new_candidates):
        star = AllStar.get_apogee_id(session, apogee_id)
        data = star.apogeervdata()
        samples = JokerSamples.from_hdf5(f[apogee_id])
        
        fig = plot_diag_panel(data, samples, M1=m1, 
                              joker_params=joker_pars)
        fig.axes[0].set_title(r'{0} ($\log g = {1}$)'
                              .format(star.apogee_id, star.logg))
        fig.tight_layout()
        fig.savefig('../plots/dr15-candidates/{0}.png'.format(apogee_id), dpi=250)
        plt.close(fig)

In [ ]:
np.savetxt('../cache/dr15-candidates.txt', new_candidates, fmt='%s')